### Import and load data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
import pickle
from sklearn.cluster import KMeans

In [ ]:
data = np.load('data/MNIST/data_all.npz')
trainv = data['trainv']
testv = data['testv']
trainlab = data['trainlab']
testlab = data['testlab']

### Training and prediction

In [ ]:
# Generate clusters
n_clusters = 64
clusterCenters = []
for i in range(10):
    train_data_class = trainv[trainlab == i]
    kmeans = KMeans(n_clusters=n_clusters, random_state=420)
    kmeans.fit(train_data_class)
    clusterCenters.append(kmeans.cluster_centers_)

clusterCenters = np.concatenate(clusterCenters)
print(clusterCenters.shape)

#### Single closest neighbour classification

In [ ]:
def predict(cluster_vec, test_vec):
    # Compute Euclidean distances
    distances = cdist(test_vec, cluster_vec,metric='euclidean')
    # Find the nearest neighbor
    nearest_idx = np.argmin(distances,axis=1)
    return np.floor(nearest_idx / 64).astype(int)

Prediction ran for 1.3s

In [ ]:
total_tests = testv.shape[0]
predictions = []
for i in range(int(total_tests/1000)):
    test_batch = testv[i*1000:(i+1)*1000]

    batch_pred = predict(clusterCenters, test_batch)

    predictions.append(batch_pred)

predictions = np.concatenate(predictions)
predictions.shape

In [ ]:
# Save predictions to file
with open('predictions/mnist_NN_cluster_predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)

#### K closest neighbour classification

In [ ]:
def KNN_predict(cluster_vec, test_vec, K):
    # Compute Euclidean distances
    distances = cdist(test_vec, cluster_vec,metric='euclidean')
    # Find the K indexes for shortest distances
    nearest_idx = np.argpartition(distances, kth=K-1, axis=1)[:, :K]

    K_predicted_labels = np.floor(nearest_idx / 64).astype(int)

    predicted_label_counts = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=1, arr=K_predicted_labels)

    predictions = predicted_label_counts.argmax(axis=1)

    return predictions

Prediction ran for 1.3s

In [ ]:
total_tests = testv.shape[0]
predictions_7NN = []
for i in range(int(total_tests/1000)):
    test_batch = testv[i*1000:(i+1)*1000]

    batch_pred = KNN_predict(clusterCenters, test_batch, 7)

    predictions_7NN.append(batch_pred)

predictions_7NN = np.concatenate(predictions_7NN)
predictions_7NN.shape

In [ ]:
# Save predictions to file
with open('predictions/mnist_7NN_cluster_predictions.pkl', 'wb') as f:
    pickle.dump(predictions_7NN, f)

### Analysis

#### Load predictions

In [ ]:
# Load predictions
with open('predictions/mnist_NN_cluster_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)

In [ ]:
# Load predictions
with open('predictions/mnist_7NN_cluster_predictions.pkl', 'rb') as f:
    predictions_7NN = pickle.load(f)

#### Plot

In [ ]:
def get_conf(pred):
    if pred.shape != testlab.shape:
        assert("shapes dont match")

    conf_matrix = np.zeros((10,10),dtype=int)
    for i in range(pred.shape[0]):
        conf_matrix[testlab[i],pred[i]] += 1

    labels = np.arange(10).astype(str)

    total_sum = np.sum(conf_matrix)
    diag_sum = np.trace(conf_matrix)
    err_rate = round((total_sum-diag_sum)/total_sum,4)*100

    plt.figure(figsize=(6, 5))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels)

    plt.xlabel("Predicted Number", fontsize = 12)
    plt.ylabel("True Number", fontsize = 12)
    plt.title(f"Confusion Matrix - Error Rate: {err_rate}%")
    plt.tight_layout()
    plt.show()

In [ ]:
get_conf(predictions)
get_conf(predictions_7NN)

In [ ]:
if predictions.shape != testlab.shape:
    assert("shapes dont match")

conf_matrix = np.zeros((10,10),dtype=int)
for i in range(predictions_7NN.shape[0]):
    conf_matrix[testlab[i],predictions_7NN[i]] += 1

labels = np.arange(10).astype(str)

total_sum = np.sum(conf_matrix)
diag_sum = np.trace(conf_matrix)
err_rate = round((total_sum-diag_sum)/total_sum,4)*100

from matplotlib.colors import LogNorm

# True Positives: diagonal
true_positives = np.diag(conf_matrix)

# Total samples per class (sum of rows)
class_totals = np.sum(conf_matrix, axis=1)

# Avoid division by zero
class_accuracy = np.divide(true_positives, class_totals, out=np.zeros_like(true_positives, dtype=float), where=class_totals!=0)

# Convert to percentage
class_accuracy_percent = np.round(class_accuracy * 100, 2)
class_error_percent = 100 - class_accuracy_percent


# Shifted matrix for log color
conf_matrix_safe = conf_matrix + 1

fig, ax = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'width_ratios': [3, 1]})

# Confusion Matrix
sns.heatmap(conf_matrix_safe, annot=conf_matrix, fmt='d',
            cmap='Blues', norm=LogNorm(vmin=1, vmax=conf_matrix_safe.max()),
            xticklabels=labels, yticklabels=labels, ax=ax[0])

ax[0].set_xlabel("Predicted Number", fontsize=12)
ax[0].set_ylabel("True Number", fontsize=12)
ax[0].set_title(f"Confusion Matrix\nError Rate: {err_rate}%", fontsize=13)

# Bar Plot: Accuracy per class
ax[1].barh(labels, class_accuracy_percent, color='mediumseagreen', label='Correct')
ax[1].barh(labels, class_error_percent, left=class_accuracy_percent, color='salmon', label='Incorrect')

ax[1].set_xlim(0, 100)
ax[1].set_xlabel("Classification %")
ax[1].set_title("Per-Class Accuracy")
ax[1].legend(loc='upper left')
ax[1].invert_yaxis()  

plt.tight_layout()
plt.show()

In [ ]:
wrong_idx = []
for i in range(predictions.shape[0]):
    if predictions[i] != testlab[i]:
        wrong_idx.append(i)

wrong_idx_7NN = []
for i in range(predictions_7NN.shape[0]):
    if predictions_7NN[i] != testlab[i]:
        wrong_idx_7NN.append(i)

In [ ]:
for i in range(4):
    im_idx = wrong_idx_7NN[i]

    print(f"Predicted = {predictions[im_idx]}, Predicted_7NN = {predictions_7NN[im_idx]}, True label = {testlab[im_idx]}")
    x = testv[im_idx, :].reshape((28, 28))
    plt.imshow(x, cmap="gray")
    plt.show()